In [1]:
import numpy as np
from scipy import signal, misc, ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Computer Vision Recap - Motion and Stereo

<br><br><br>
## Lecture 7: Local Methods

**Goal:**  Given an image sequence $f(x,y,t)$, find a displacement vector field (optical flow) between images $u(x,y,z)$ and $v(x,y,z)$

### Grey Value Constancy assumption

$$ f(x,y,z) = f(x+u,y+v,z+1) $$

Which can be linearized as:
$$ f(x+u,y+v,z+1) \approx f(x,y,z) + u\cdot f_x + v\cdot f_y + f_z $$

and the new grey value constancy assumption (OFC) becomes:
$$ u\cdot f_x + v\cdot f_y + f_z = 0 $$

### Aparture Problem

The OFC equation gives us one equation and two unkowns $u,v$ to solve.

WE can only solve the flow component that is parallel to the spatial gradient

$$ f_x u + f_y v + f_z = \begin{bmatrix}u\\v\end{bmatrix}^T \nabla f + f_z = 0 $$

This means that additional flow orthogonal to $\nabla f^\perp$ without violating the OFC

The normal flow can be rewriten as:
$$ \left( \begin{array}{l} u_{n} \\ v_{n} \end{array} \right) = \left[\left(\begin{array}{l} u \\ v \end{array}\right)^T \frac{\nabla f}{|\nabla f|}\right] \frac{\nabla f}{|\nabla f|} = -\frac{f_{z}}{|\nabla f|} \frac{\nabla f}{|\nabla f|} $$

### Lucas and Kanade (Least Square Fit)

**Goal:** Minimize the local energy within some neighborhood $B_\rho(x_0,y_0)$ of radius $\rho$:

$$E(u, v)=\frac{1}{2} \int_{B_{\rho}\left(x_{0}, y_{0}\right)}\left(f_{x} u+f_{y} v+f_{z}\right)^{2} dx dy$$

Next, computing partial derivatives w.r.t. $u$ and $v$ gives:
$$ \frac{\partial E}{\partial u}=\int_{B_{\rho}} f_{x}\left(f_{x} u+f_{y} v+f_{z}\right) d x d y \stackrel{!}{=} 0 $$
$$ \frac{\partial E}{\partial v}=\int_{B_{\rho}} f_{y}\left(f_{x} u+f_{y} v+f_{z}\right) d x d y \stackrel{!}{=} 0 $$

writing in matrix form:
$$\left(\begin{array}{cc}\int_{B_{\rho}} f_{x}^{2} d x d y & \int_{B_{\rho}} f_{x} f_{y} d x d y \\ \int_{B_{\rho}}^{2} f_{x} f_{y} d x d y & \int_{B_{\rho}} f_{y}^{2} d x d y\end{array}\right)\left(\begin{array}{c}u \\ v\end{array}\right)=\left(\begin{array}{c}-\int_{B_{\rho}} f_{x} f_{z} d x d y \\ -\int_{B_{\rho}} f_{y} f_{z} d x d y\end{array}\right)$$

Next we can replace the "hard" window $B_\rho$ by a smooth convolution with a Gaussian $K_\rho$
$$ J_{\rho} \left(\begin{array}{c}u \\ v\end{array}\right) = \left(\begin{array}{cc}{K_{\rho} * f_{x}^{2}} & {K_{\rho} * f_{x} f_{y}} \\ {K_{\rho} * f_{x} f_{y}} & {K_{\rho} * f_{y}^{2}}\end{array}\right) \left(\begin{array}{c}u \\ v\end{array}\right) = \left(\begin{array}{l} -K_{\rho} *\left(f_{x} f_{z}\right) \\ -K_{\rho} *\left(f_{y} f_{z}\right) \end{array}\right) $$

where $J_\rho$ is the Structure Tensor

The Lucas-Kanade method solves a $2\times2$ linear system of equations for each pixel.

Lucas-Kanade assumes a constant displacement withing a local neighbourhood

**Note:**
- rank($J_\rho$)=0 (trace$\approx$0): Flat area. No flow can be determined in this case.
- rank($J_\rho$)=1 (det$\approx$0): Edge. Aperture problem. One can only compute the normal flow. 
- rank($J_\rho$)=2: Corner. Full flow be computed, since $J_\rho$ is full rank.

** Advantages **:
- Simple and fast to compute

** Disadvantages **:
- Does not allow flow field computation at all locations
- Does not work at non-translatory motion and at flow discontinuities

### The Approach of Biguen (Total Least Square Fit)

The energy function can be written as:
$$E(\tilde{\boldsymbol{w}})=\int_{B_{\rho}\left(x_{0}, y_{0}\right)}\left(f_{x} \tilde{w}_{1}+f_{y} \tilde{w}_{2}+f_{z} \tilde{w}_{3}\right)^{2} d x d y$$

with $u=\frac{\tilde{w}_{1}}{\tilde{w}_{3}}, \quad v=\frac{\tilde{w}_{2}}{\tilde{w}_{3}}$

which is equal to
$$\begin{aligned} E(\tilde{\boldsymbol{w}}) &:=\int_{B_{\rho}}\left(\tilde{\boldsymbol{w}}^{\top} \nabla_{3} f\right)^{2} d x d y \\ &=\int_{B_{\rho}} \tilde{\boldsymbol{w}}^{\top} \nabla_{3} f \nabla_{3} f^{\top} \tilde{\boldsymbol{w}} d x d y \\ &=\boldsymbol{w}^{\top}\left(\int_{B_{\rho}} \nabla_{3} f \nabla_{3} f^{\top} d x d y\right) \tilde{\boldsymbol{w}} \end{aligned}$$

The vector $w$ that minimizes $E(w)$ is the same as the normalised eigenvector to the smallest eigenvalue of the spatiotemporal structure tensor $J_\rho$:
$$\begin{aligned} J_{\rho} &:=K_{\rho} *\left(\nabla_{3} f \nabla_{3} f^{\top}\right) \\ &=\left(\begin{array}{ccc}K_{\rho} *\left(f_{x}^{2}\right) & K_{\rho} *\left(f_{x} f_{y}\right) & K_{\rho} *\left(f_{x} f_{z}\right) \\ K_{\rho} *\left(f_{x} f_{y}\right) & K_{\rho} *\left(f_{y}^{2}\right) & K_{\rho} *\left(f_{y} f_{z}\right) \\ K_{\rho} *\left(f_{x} f_{z}\right) & K_{\rho} *\left(f_{y} f_{z}\right) & K_{\rho} *\left(f_{z}^{2}\right)\end{array}\right) \end{aligned}$$




**Note:**
- rank($J_\rho$)=3: OFC is violated
- rank($J_\rho$)=2: Full flow be computed, since $J_\rho$ is full rank. In this pixel we have a corner
- rank($J_\rho$)=1: Aperture problem. One can only compute the normal flow. 
- rank($J_\rho$)=0: No flow can be determined in this case.

<br><br><br>
## Lecture 8: Motion Models, Tracking and Feature Matching

### The Affine Lucas and Kanade Method

**Goal:** Instead of restricting all the neighborhood to have the same displacement, we now allow a affine change in the displacement of neighbourhood pixels

Lets consider the following parametrization:

$$\boldsymbol{w}=\left(\begin{array}{c}u \\ v \\ 1\end{array}\right)=\left(\begin{array}{c}a x+b y+c \\ d x+e y+f \\ 1\end{array}\right)=\underbrace{\left(\begin{array}{ccccccc}x & y & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & x & y & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 1\end{array}\right)}_{M} \underbrace{\left(\begin{array}{c}a \\ b \\ c \\ d \\ e \\ f \\ 1\end{array}\right)}_{\boldsymbol{p}}$$

The OFC can then be rewritten as:
$$f_{x} u+f_{y} v+f_{z}=\nabla_{3} f^{\top} \boldsymbol{w}=\nabla_{3} f^{\top} M \boldsymbol{p}=\left(M^{\top} \nabla_{3} f\right)^{\top} \boldsymbol{p}=r^{\top} \boldsymbol{p}$$

And the energy function to be minimized is:
$$\begin{aligned} E(\boldsymbol{p}) &= \int_{B_{\rho}}\left(\boldsymbol{p}^{\top} \boldsymbol{r}\right)^{2} d x d y \\ &= \boldsymbol{p}^{\top}\left(\int_{\boldsymbol{B}_{\rho}} \mathbf{r} \mathbf{r}^{\top} d x d y\right) \boldsymbol{p} \\ &=\boldsymbol{p}^{\top} \hat{J}_{\rho} \boldsymbol{p} \end{aligned}$$

The minimum requires the derivatives w.r.t. all 6 parameters to be zero $\frac{\partial E}{\partial a}, \frac{\partial E}{\partial b}, \dots, \frac{\partial E}{\partial f} = 0$













### The KLT Feature Tracker

1. Select features to be tracked - neighbourhood around corners
2. Compute inter-frame displacements for each feature - Translational (nonlinear) Lucas-Kanade method
3. Monitor features at new location - compute cumulative error by evaluating OFC (Affine Lucas-Kanade)
$\rightarrow$ discard features that could not be tracked
4. Go to (2), but only consider features not discarded in (3) some approaches also allow to add new features at this point

#### 1. Feature Selection

Select points by thresholding the smallest eigenvalue of the structure tensor. This comes down to detecting and tracking corners

Perform feature thinning: remove features until feature neighborhood does not overlap anymore 


#### 2. Tracking (Nonlinear Lucas Kanade)

Here, a nonlinear Lucas-Kanade method is used (with increment linearization and backward-registration), since the linearized OFC is tipically not sufficient for an accurate tracking.

The nonlinear OFC is:
$$\begin{aligned} E\left(u^{k+1}, v^{k+1}\right) &=\int_{B_{\rho}}\left(f\left(x+u^{k+1}, y+v^{k+1}, z+1\right)-f(x, y, z)\right)^{2} d x d y \\ &=\int_{B_{\rho}}\left(f\left(x+u^{k}+d u^{k}, y+v^{k}+d v^{k}, z+1\right)-f(x, y, z)\right)^{2} d x d y \end{aligned}$$

which after linearization of $du^k, dv^k$ becomes:
$$\begin{aligned} E\left(d u^{k}, d v^{k}\right)=\int_{B_{\rho}}(& f_{x}\left(x+u^{k}, y+v^{k}, z+1\right) d u^{k} \\+& f_{y}\left(x+u^{k}, y+v^{k}, z+1\right) d v^{k} \\+&\left.f_{z}\left(x+u^{k}, y+v^{k}, z+1\right)\right) \end{aligned}$$
with initial flow $(u^0, v^0)\top= (0,0)\top$ and update equation
$$\left(u^{k+1}, v^{k+1}\right)^{\top}=\left(u^{k}, v^{k}\right)^{\top}+\left(d u^{k}, d v^{k}\right)^{\top}$$

However, $u^k$ and $v_k$ tipically have subpixel precision, requiring us to perform a interpolation to be able to evaluate $f(x+u^k,y+v^k,z+1)$

<img src="./images/backwardreg.png" width=400>

#### 3. Monitoring (Affine Lucas-Kanade)

**Goal:** Detect when we lose track of a feature. This can be verified by evaluating the OFC violation:

$$ e = \int_{B_\rho} \left( f(x+u,y+v,z+1) - f(x,y,1) \right)^2 dx dy $$

which is computed using an affine Lucas-Kanade motion model, which discriminates much more the errors, making it easier to discard "bad" features. 





#### Feature Matching

Instead of tracking feature points over several frames it is better to compute a globally parametrized feature.

1. Compute feature descriptor SIFT for both images
2. Compare feature vectors in first image to all feature vectors in second image. The distance between feature $a$ and $b$ is the sum of squared differences (SSD): $diff(a,b) = \sum(a_k,b_k)^2$
3. Discard matches when cost ration between best and second best < threshold

**Advantages:** 
- Allows to estimate global motion models with a few parameters
- Sparse results, but very accurate

**Disadvantages:** 
- Yields a non-dense flow field which are not usefull for all tasks
- Produces outliers, that must be handled 


<br><br><br>
## Lecture 9: Variational Methods

### Horn and Schunck method

Variational method, in which the energy functional contain two assumptions:

1. Grey value constancy assumption (OFC)
2. Flow field Smoothness 

$$E(u,v) = \int_{\Omega} \left( (f_x u + f_y v + f_z)^2 + \alpha (|\nabla u|^2 + |\nabla v|^2) \right) dx dy$$

**Filling-in-effect**: This enables us to generate dense flow fields even at locations where no flow estimation is possible ($|\nabla f|\approx 0$). The information is propagated from the neighborhood


Calculus of Variations: 

Minimizing the energy functional
$$ E(u,v) = \int_\Omega F(x,y,u,v,u_x,u_y,v_x,v_y) dx dy $$
satisfies the Euler-Lagrande Equation (necessary conditions)
$$ \begin{align} & F_u - \partial_x F_{u_x} - \partial_y F_{u_y} = 0 \\ & F_v - \partial_x F_{v_x} - \partial_y F_{v_y} = 0\end{align}$$
with boundary conditions:
$$ n \cdot (F_{u_x},F_{u_y})=0, \quad\quad n \cdot (F_{v_x},F_{v_y})=0$$


which results in:
$$ \begin{align} f_x (f_x u + f_y v + f_z) - \alpha \Delta u = 0 \\ f_y (f_x u + f_y v + f_z) - \alpha \Delta v = 0 \end{align} $$
with boundary conditions
$$ n \cdot \nabla u = 0 \quad\quad n \cdot \nabla v =0 $$

which needs to be solved for all $u_{i,j}$ and $v_{i,j}$ on the image




### THe Jacobi Method

Solve $Ax=b$ iteratively. If the matrix $A$ can be decomposed into a diagonal and rest elements: $ A=D-N $, then the problem can be solved as:
$$ x^{k+1} = D^{-1} (Nx^k +b) $$















<br><br><br>
## Lecture 10: Camera Geometry


### Camera Projective Mapping

The pinhole camera model:

<img src="./images/cameramodel.png" width=400>

The nonlinear projective geometry equation is simply
$$ \frac{x}{X} = \frac{y}{Y} = \frac{f}{Z}  \quad \Rightarrow \quad x = \frac{f.X}{Z}, \quad y = \frac{f.Y}{Z}$$

which can be written in a linear form by representing in homogeneous coordinates
$$ \begin{bmatrix} \tilde x \\ \tilde y \\ \lambda \end{bmatrix} = \begin{bmatrix} Zx\\Zy\\Z \end{bmatrix} = \begin{bmatrix}f&0&0&0\\0&f&0&0\\0&0&1&0\end{bmatrix} \begin{bmatrix} X\\Y\\Z\\1 \end{bmatrix}$$
which can be written in a compact form as
$$ \lambda \tilde m = P_f \tilde M $$

General projective Mapping in homogeneous coordinates
<img src="./images/cameramatrix.png" width=600>

which has 12 entries but 11 degrees of freedom (free scaling parameter, f is absorbed into $k_u$ and $k_v$)


- The extrinsic matrix contains 3 translational and 3 rotational parameters 
- The intrinsic matrix contains parameters concerning image coordinate system, pixel density and angle between cordinate axes.

The projection equation is then given by: 
<img src="./images/cameraprojmapp.png" width=400>

**Problems:** $\tilde w$ might be close to zero making the problem ill-posed

<br>

### Camera Affine Mapping

Considering $p_{3,1}, p_{3,2}, p_{3,3} = 0$, leads to a linear affine mapping from 3D to 2D.

<img src="./images/cameraaffmapp.png" width=400>

- Reduces the degrees of freedom from 11 to 6
- Problem is now always well-posed

- this model is reasonable simplification if the relative depth variation of the object is smal

<br>

### Orthografic Mapping

- completely ignores the depth information
- still popular for specific applications, e.g. shape from shading

<br><br><br>
## Lecture 11: Epipolar Geometry, Stereo Matching

### Epipolar Geometry

<img src="./images/stereogeometry.png" width=400>

Disparity: distance between conjugated points, if both images are superposed

If both cameras are orthoparallel, then the depth can be estimated by:
$$ Z = \frac{b\;f}{x_1-x_2} = \frac{b\;f}{d} $$
where $b$ is the baseline, $f$ the focal length and $d$ the disparity

**Epipolar constraint:**
$$ \tilde m_2^\top \; F \; \tilde m_1 = 0 $$

where the fundamental matrix can be computed via
$$ F = A_{2,int}^{-T} \; [t]_x \; R \; A_{1,int}^{-1}$$
with $R$ and $t$ obtained from $A=A_{2,ext}\, A_{1,ext}^{-1}$

F is rank 2 and not invertible because of the skew-symmetric matrix $[t]_x$

<br><br>

### Estimation of Fundamental matrix

<img src="./images/Festimation.png" width=500>

Problems:
- Trivial solution $f=0$ always works.
- Problem is overdetermined for N>8

**Solution** minimize the square deviation of the sum of N epipolar constraints

$$ E(f) = \sum^N_{i=1} (s_i^\top f)^2 = f^\top (\sum^N_{i=1} s_i s_i^\top )f = f^\top M f $$
with explicit constraint $|f|=1$ to avoid trivial solution.

The solution to this problem is equal to the normalised eigenvector to the smallest eigenvalue of the $9\times9$ matrix M



### The Stereo Problem

#### Correlation-Based Methods

this is the most simple method of optical flow.

For each pixel, try all displacements in the search space that gives the highest correlation coefficient

<img src="./images/corrcoef.png" width=500>

which is unreliable in flat regions but simple and fast

#### Variational Methods (only orthonormal case)

Minimize the energy functional

$$ E(u) = \int_{\Omega} \left( (f_2(x+u,y)-f_1(x,y))^2 + \alpha |\nabla u|^2 \right) dx dy $$

Since disparities might be large, linearization is not appropriate. This implies that the energy functional will not be convex.
